In [1]:
import torch
import triton
import triton.language as tl

DEVICE = 'cuda'

In [2]:
@triton.jit
def scalar_mult_duplicate(x_ptr, output_ptr, n_elements, BLOCK_SIZE: tl.constexpr):
    pid = tl.program_id(axis=0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    mask = offsets < n_elements

    x = tl.load(x_ptr + offsets, mask=mask).reshape([1, BLOCK_SIZE]).broadcast_to(2, BLOCK_SIZE)
    output = 4 * x

    first_row_idx = block_start + tl.arange(0, BLOCK_SIZE)
    second_row_idx = block_start + tl.arange(0, BLOCK_SIZE) + n_elements
    first_row_mask = first_row_idx < n_elements
    second_row_mask = second_row_idx < (n_elements * 2)

    broadcasted_offsets = tl.trans(tl.join(first_row_idx, second_row_idx))
    broadcasted_mask = tl.trans(tl.join(first_row_mask, second_row_mask))

    tl.store(output_ptr + broadcasted_offsets, output, mask=broadcasted_mask)

In [3]:
def mult(x: torch.Tensor):
    output = torch.empty([2, x.shape[0]], device=DEVICE)
    n_elements = x.numel()
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
    scalar_mult_duplicate[grid](x, output, n_elements, BLOCK_SIZE=1024)
    return output

In [4]:
!rm -rf dump.txt

In [5]:
%env MLIR_ENABLE_DUMP=1
%env MLIR_DUMP_PATH=dump.txt
!rm -rf ~/.triton

torch.manual_seed(0)
size = 98432
x = torch.rand(size, device=DEVICE)
output_torch = 4 * x.broadcast_to([2, size])
output_triton = mult(x)
print(output_torch)
print(output_triton)
print(f'The maximum difference between torch and triton is '
      f'{torch.max(torch.abs(output_torch - output_triton))}')

env: MLIR_ENABLE_DUMP=1
env: MLIR_DUMP_PATH=dump.txt


tensor([[1.5962, 2.0667, 0.0997,  ..., 0.1694, 2.0397, 2.9107],
        [1.5962, 2.0667, 0.0997,  ..., 0.1694, 2.0397, 2.9107]],
       device='cuda:0')
tensor([[1.5962, 2.0667, 0.0997,  ..., 0.1694, 2.0397, 2.9107],
        [1.5962, 2.0667, 0.0997,  ..., 0.1694, 2.0397, 2.9107]],
       device='cuda:0')
The maximum difference between torch and triton is 0.0
